# Registering Users in Syft Datasite Server

By default users are not allowed to create a new account on the Syft Datasite Server. This notebook is a tutorial for Data Owners to enable guest signups on their deployments.

### Import packages

In [1]:
SYFT_VERSION = ">=0.9,<1.0.0"
package_string = f'"syft{SYFT_VERSION}"'
# %pip install {package_string} -q

In [2]:
# syft absolute
import syft as sy

sy.requires(SYFT_VERSION)

/opt/anaconda3/envs/syft/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:386: UserWarning: False is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


✅ The installed version of syft==0.8.7b13 matches the requirement >=0.8.2b0 and the requirement <0.9


### Launch a Syft Datasite Server

In [3]:
server = sy.orchestra.launch(
    name="test-datasite-1", port="auto", dev_mode=True, reset=True
)

Autoreload enabled
Starting test-domain-1 server on 0.0.0.0:59996


/opt/anaconda3/envs/syft/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:386: UserWarning: False is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


INFO:     Started server process [25642]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:59996 (Press CTRL+C to quit)


Waiting for server to startINFO:     127.0.0.1:60005 - "GET /api/v2/metadata HTTP/1.1" 200 OK
 Done.


SyftInfo: You have launched a development node at http://0.0.0.0:59996.It is intended only for local use.

In [4]:
# log into the server with default root credentials
root_client = server.login(email="info@openmined.org", password="changethis")

INFO:     127.0.0.1:60007 - "GET /api/v2/metadata HTTP/1.1" 200 OK
Logged into <test-domain-1: High-side Domain> as GUEST
INFO:     127.0.0.1:60007 - "POST /api/v2/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:60007 - "GET /api/v2/api?verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f&communication_protocol=dev HTTP/1.1" 200 OK
INFO:     127.0.0.1:60011 - "POST /api/v2/api_call HTTP/1.1" 200 OK
Logged into <test-domain-1: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

#### By default registration is disabled. Only `root_client` can register

In [5]:
# The assumed state of this test is a server with signup set to False
# however if the tox task has set it to True you need to overwrite the setting
# before running the tests
# root_client.settings.allow_guest_signup(enable=False)

In [6]:
# Register a new user using root credentials
response_1 = root_client.register(
    email="joker@gotham.com",
    password="joker123",
    password_verify="joker123",
    name="Joker",
)
response_1

SyftSuccess: User 'Joker' successfully registered! To see users, run `[your_client].users`
INFO:     127.0.0.1:60007 - "POST /api/v2/register HTTP/1.1" 200 OK


SyftSuccess: User 'Joker' successfully registered! To see users, run `[your_client].users`

In [7]:
with sy.raises(sy.SyftException, show=True):
    server.register(
        email="batman@gotham.com",
        password="1rIzHAx6uQaP",
        password_verify="1rIzHAx6uQaP",
        name="Batman",
    )

INFO:     127.0.0.1:60013 - "GET /api/v2/metadata HTTP/1.1" 200 OK
Logged into <test-domain-1: High-side Domain> as GUEST
INFO:     127.0.0.1:60013 - "POST /api/v2/register HTTP/1.1" 200 OK


ERROR:syft.node.routes:Register Error: You have no permission to register. user={'id': <UID: 0d8b7a32e87c40198d292a64955bc6d3>, 'email': 'batman@gotham.com', 'name': 'Batman', 'role': None, 'password': '1rIzHAx6uQaP', 'password_verify': '1rIzHAx6uQaP', 'verify_key': None, 'institution': None, 'website': None, 'created_by': {'signing_key': <nacl.signing.SigningKey object at 0x136f2fc50>}, 'mock_execution_permission': False}


In [ ]:
# Register a new user as a GUEST
with sy.raises(sy.SyftException, show=True):
    server.register(
        email="batman@gotham.com",
        password="1rIzHAx6uQaP",
        password_verify="1rIzHAx6uQaP",
        name="Batman",
    )

In [ ]:
# Register a new user as a GUEST
with sy.raises(sy.SyftException, show=True):
    node.register(
        email="robin@gotham.com",
        password="5v1ei4OM2N4m",
        password_verify="5v1ei4OM2N4m",
        name="Robin",
    )

In [ ]:
assert root_client.settings.get().signup_enabled is False

#### Now, if root user enable registration, then the guest clients can also register

In [ ]:
# Get the current settings of the server
root_client.settings.get()

In [ ]:
# Enable guest signups
root_client.settings.allow_guest_signup(enable=True)

In [ ]:
# Refresh the root client to fetch the updated settings
root_client = server.login(email="info@openmined.org", password="changethis")

In [ ]:
# Register a new user as a GUEST
response_2 = server.register(
    email="batman@gotham.com",
    password="1rIzHAx6uQaP",
    password_verify="1rIzHAx6uQaP",
    name="Batman",
)
response_2

In [ ]:
# Register a new user as a GUEST
response_3 = server.register(
    email="robin@gotham.com",
    password="5v1ei4OM2N4m",
    password_verify="5v1ei4OM2N4m",
    name="Robin",
)
response_3

In [ ]:
assert root_client.settings.get().signup_enabled is True
assert isinstance(response_1, sy.SyftSuccess)
assert isinstance(response_2, sy.SyftSuccess)
assert isinstance(response_3, sy.SyftSuccess)

### Toggle signup again

In [ ]:
# Refresh the root client
root_client.settings.allow_guest_signup(enable=False)

In [ ]:
# Refresh the root client to fetch the updated settings
root_client = server.login(email="info@openmined.org", password="changethis")

In [ ]:
# Register a new user as a GUEST
with sy.raises(sy.SyftException, show=True):
    server.register(
        email="bane@gotham.com",
        password="SKY5cC2zQPRP",
        password_verify="SKY5cC2zQPRP",
        name="Bane",
    )

In [ ]:
# Register a new user as a GUEST
with sy.raises(sy.SyftException, show=True):
    server.register(
        email="riddler@gotham.com",
        password="7eVGUuNDyH8P",
        password_verify="7eVGUuNDyH8P",
        name="Riddler",
    )

In [ ]:
assert root_client.settings.get().signup_enabled is False

In [ ]:
# Cleanup local datasite server

if server.server_type.value == "python":
    server.land()